## TMA4215 - Project #1 ## 
*27.09.2024*

*Candidate ID*

In this project I have used numerical methods for Fourier series based signal processing.

In [5]:
import numpy as np
import matplotlib.pyplot as plt
from numba import jit, njit

#### Task 1: The (Discrete) Fourier Transform

#### a)

Considering the functions $e^{2\pi ikx}$, $k \in \mathbb{Z}$, $x \in \mathbb{T}$. I am going to below prove that for any $k, h \in \mathbb{Z}$ we have

$$
\left\langle e^{2\pi i k \cdot}, e^{2\pi i h \cdot} \right\rangle = 
\begin{cases} 
1 & \text{if } k = h \\
0 & \text{else}
\end{cases}
$$


The inner product of two functions $f(x)$ and $g(x)$ on the interval $[0, 1)$ is defined as:

$$
\langle f, g \rangle = \int_0^1 f(x) \overline{g(x)} \, dx
$$

For our case, $f(x) = e^{2\pi i k x}$ and $g(x) = e^{2\pi i h x}$. The complex conjugate of $g(x)$ is:

$$
\overline{g(x)} = e^{-2\pi i h x}
$$

Thus, the inner product becomes:

$$
\langle e^{2\pi i k x}, e^{2\pi i h x} \rangle = \int_0^1 e^{2\pi i k x} \cdot e^{-2\pi i h x} \, dx = \int_0^1 e^{2\pi i (k - h) x} \, dx
$$

We have two cases:

1. **$k = h$:**

$$
\int_0^1 e^{2\pi i (k - h) x} \, dx = \int_0^1 e^{0} \, dx = \int_0^1 1 \, dx = 1
$$

So, the inner product is 1 when $k = h$.

2. **$k \neq h$:**

In this case, let $m = k - h \neq 0$. We need to evaluate:

$$
\int_0^1 e^{2\pi i m x} \, dx
$$

The antiderivative of $e^{2\pi i m x}$ is:

$$
\int e^{2\pi i m x} \, dx = \frac{e^{2\pi i m x}}{2\pi i m}
$$

Evaluating this from $0$ to $1$ gives:

$$
\left[ \frac{e^{2\pi i m x}}{2\pi i m} \right]_0^1 = \frac{e^{2\pi i m \cdot 1} - e^{2\pi i m \cdot 0}}{2\pi i m} = \frac{e^{2\pi i m} - 1}{2\pi i m}
$$

Since $e^{2\pi i m} = 1$ for any integer $m$, we have:

$$
\frac{1 - 1}{2\pi i m} = 0
$$

So, the inner product is $0$ when $k \neq h$.

$\square$


#### b)

Now considering the functions of the form $\sqrt{2} \sin(2\pi mx)$, $m = 1, 2, \ldots$, $\cos(2\pi 0 x)$, and $\sqrt{2} \cos(2\pi nx)$, $n = 1, 2, \ldots$, $x \in \mathbb{T}$. I want to prove that these form an orthonormal system.

1. **Orthogonality between sine and cosine functions:**

The inner product of $\sqrt{2} \sin(2\pi n x)$ and $\sqrt{2} \cos(2\pi m x)$ is given by:

$$
\langle \sqrt{2} \sin(2\pi n x), \sqrt{2} \cos(2\pi m x) \rangle = 2 \int_0^1 \sin(2\pi n x) \cos(2\pi m x) \, dx
$$

Using the trigonometric identity:

$$
\sin A \cos B = \frac{1}{2} [\sin(A + B) + \sin(A - B)],
$$

we get:

$$
2 \int_0^1 \sin(2\pi n x) \cos(2\pi m x) \, dx = \int_0^1 [\sin(2\pi (n+m) x) + \sin(2\pi (n-m) x)] \, dx
$$

Both integrals of $\sin(2\pi k x)$ over the interval $[0, 1]$ are zero because the sine function completes an integer number of periods, and its average value over a full period is zero. Therefore, the inner product is also zero. Hence:

$$
\langle \sqrt{2} \sin(2\pi n \cdot), \sqrt{2} \cos(2\pi m \cdot) \rangle = 0, \quad n \in \{1, 2, \ldots\}, \, m \in \{0, 1, \ldots\}
$$

2. **Orthogonality and normality between two sine functions:**

The inner product of $\sqrt{2} \sin(2\pi n x)$ and $\sqrt{2} \sin(2\pi m x)$ is:

$$
\langle \sqrt{2} \sin(2\pi n x), \sqrt{2} \sin(2\pi m x) \rangle = 2 \int_0^1 \sin(2\pi n x) \sin(2\pi m x) \, dx
$$

Using the identity:

$$
\sin A \sin B = \frac{1}{2} [\cos(A - B) - \cos(A + B)],
$$

we get:

$$
2 \int_0^1 \sin(2\pi n x) \sin(2\pi m x) \, dx = \int_0^1 [\cos(2\pi (n-m) x) - \cos(2\pi (n+m) x)] \, dx
$$

The integral of $\cos(2\pi k x)$ over $[0, 1]$ is zero for any integer $k \neq 0$. For $k = 0$ (when $n = m$), the integral of $1$ over $[0, 1]$ is $1$. Hence, the inner product is $1$ if $n = m$ and $0$ otherwise. So:

$$
\langle \sqrt{2} \sin(2\pi n \cdot), \sqrt{2} \sin(2\pi m \cdot) \rangle = 
\begin{cases} 
0 & \text{if } m \neq n \\
1 & \text{if } m = n 
\end{cases}, \quad m, n \in \{1, 2, \ldots\}
$$

3. **Orthogonality and normality between two cosine functions:**

The inner product of $\sqrt{2} \cos(2\pi n x)$ and $\sqrt{2} \cos(2\pi m x)$ is:

$$
\langle \sqrt{2} \cos(2\pi n x), \sqrt{2} \cos(2\pi m x) \rangle = 2 \int_0^1 \cos(2\pi n x) \cos(2\pi m x) \, dx
$$

Using the identity:

$$
\cos A \cos B = \frac{1}{2} [\cos(A + B) + \cos(A - B)],
$$

we get:

$$
2 \int_0^1 \cos(2\pi n x) \cos(2\pi m x) \, dx = \int_0^1 [\cos(2\pi (n + m) x) + \cos(2\pi (n - m) x)] \, dx
$$

The integral of $\cos(2\pi k x)$ over $[0, 1]$ is zero for any integer $k \neq 0$. For $k = 0$ (when $n = m$), the integral of $1$ over $[0, 1]$ is $1$. For $n = m = 0$, the integral of $1$ over $[0, 1]$ multiplied by $2$ is $2$. Hence, the inner product is $2$ if $n = m = 0$, $1$ if $n = m \neq 0$, and $0$ otherwise. Which we can write as:

$$
\langle \sqrt{2} \cos(2\pi n \cdot), \sqrt{2} \cos(2\pi m \cdot) \rangle = 
\begin{cases} 
0 & \text{if } m \neq n \\
1 & \text{if } m = n \neq 0 \\
2 & \text{if } m = n = 0 
\end{cases}, \quad m, n \in \{0, 1, \ldots\}
$$

$\square$


#### c)

Now we introduce the following two spaces, $T_n$ and $S_n$:

$$
T_n := \text{span}(e^{-2\pi in\cdot}, \ldots, e^{2\pi in\cdot}) = \left\{ f \, \middle| \, f(x) = \sum_{k=-n}^{n} c_k e^{2\pi ikx}, \, \text{where } c_{-n}, c_{-n+1}, \ldots, c_n \in \mathbb{C} \right\}
$$

$$
S_n := \text{span}(\cos(0\cdot), \cos(2\pi\cdot), \ldots, \cos(2\pi n\cdot), \sin(2\pi\cdot), \sin(2\pi \cdot 2), \ldots, \sin(2\pi n\cdot))
$$

$$
= \left\{ f \, \middle| \, f(x) = \frac{a_0}{2} + \sum_{k=1}^{n} a_k \cos(2\pi kx) + b_k \sin(2\pi kx), \, \text{where } a_0, a_1, \ldots, a_n, b_1, \ldots, b_n \in \mathbb{R} \right\}
$$

Using the results from Item a) and b) I want to find orthonormal bases for these two spaces.

**Orthonormal basis for $T_n$:**

From Item a), we have shown that the functions $e^{2\pi i k \cdot}$ for $k = -n, \ldots, n$ form an orthonormal set on the interval $[0, 1)$. To construct an orthonormal basis for $T_n$, we consider the set:

$$
\left\{ e^{2\pi i k x} \, \middle| \, k = -n, \ldots, n \right\}
$$

Thus, the orthonormal basis for $T_n$ is:

$$
\left\{ e^{2\pi i k x} \, \middle| \, k = -n, \ldots, n \right\}
$$

**Orthonormal basis for $S_n$:**

From Item b), we have shown the orthogonality properties of the sine and cosine functions. We can use the results to form orthonormal bases for $S_n$.

- **For Cosine Functions:**

The function $\cos(0)$ is not normalized, so we use $1$ instead of $\cos(0)$. For $k = 1, \ldots, n$, the normalized cosine functions are $\sqrt{2} \cos(2\pi k x)$.

The orthonormal basis for the cosine components is:

$$
\left\{ 1, \sqrt{2} \cos(2\pi k x) \, \middle| \, k = 1, 2, \ldots, n \right\}
$$

- **For Sine Functions:**

For $k = 1, \ldots, n$, the normalized sine functions are $\sqrt{2} \sin(2\pi k x)$.

The orthonormal basis for the sine components is:

$$
\left\{ \sqrt{2} \sin(2\pi k x) \, \middle| \, k = 1, 2, \ldots, n \right\}
$$


**Proving $T_n = S_n$ Using Euler's Identity:**

Euler's identity states:

$$
e^{i\theta} = \cos(\theta) + i\sin(\theta)
$$

Using this, we can express the exponential functions in $T_n$ in terms of sines and cosines:

$$
e^{2\pi ikx} = \cos(2\pi kx) + i\sin(2\pi kx)
$$

$$
e^{-2\pi ikx} = \cos(2\pi kx) - i\sin(2\pi kx)
$$

Therefore, any function $f(x)$ in $T_n$ can be written as:

$$
f(x) = \sum_{k=-n}^{n} c_k e^{2\pi ikx} = c_0 + \sum_{k=1}^{n} c_k e^{2\pi ikx} + c_{-k} e^{-2\pi ikx}
$$

Substituting the identities:

$$
f(x) = c_0 + \sum_{k=1}^{n} \left[ c_k (\cos(2\pi kx) + i\sin(2\pi kx)) + \overline{c_k} (\cos(2\pi kx) - i\sin(2\pi kx)) \right]
$$

$$
f(x) = c_0 + \sum_{k=1}^{n} \left[ (c_k + \overline{c_k}) \cos(2\pi kx) + i(c_k - \overline{c_k}) \sin(2\pi kx) \right]
$$

Since $c_k = \overline{c_{-k}}$, we can set $c_k + \overline{c_k} = 2\Re(c_k) = a_k$ and $i(c_k - \overline{c_k}) = 2i \Im(c_k) = b_k$. This transforms the expression into:

$$
f(x) = c_0 + \sum_{k=1}^{n} a_k \cos(2\pi kx) + b_k \sin(2\pi kx)
$$

This is precisely the form of the function in $S_n$. Therefore, every function in $T_n$ can be represented as a function in $S_n$. Conversely, any function in $S_n$ can be represented in terms of the exponential form of $T_n$. Hence, $T_n = S_n$.

**Dimension of $T_n$:**

The dimension of $T_n$ is the number of linearly independent basis functions. The basis functions for $T_n$ are $e^{-2\pi inx}, e^{-2\pi i(n-1)x}, \ldots, e^{2\pi inx}$.

Since there are $2n + 1$ such basis functions, the dimension of $T_n$ is:

$$
\text{dim}(T_n) = 2n + 1
$$


#### d)

From Item c), any function $ f(x) \in S_n $ can be expressed as:

$$
f(x) = \frac{a_0}{2} + \sum_{k=1}^{n} a_k \cos(2\pi kx) + b_k \sin(2\pi kx)
$$

where $ a_0, a_1, \ldots, a_n $ and $ b_1, \ldots, b_n $ are the Fourier coefficients.

This representation decomposes any function in $S_n$ into a finite sum of simpler trigonometric functions.

The inner product on the space $S_n$ is defined as:

$$
\langle f, g \rangle = \int_{-\frac{1}{2}}^{\frac{1}{2}} f(x) g(x) \, dx
$$

We know from previously that the set of functions:

$$
\{ 1, \sqrt{2} \cos(2\pi kx), \sqrt{2} \sin(2\pi kx) \, \mid \, k = 1, 2, \ldots, n \}
$$

forms an orthonormal basis for $S_n$.

Taking the inner product of $f(x)$ with each of these basis functions allows us to isolate each Fourier coefficient.

To find $ a_k $, we take the inner product of $ f(x) $ with $ \cos(2\pi kx) $:

$$
\langle f, \cos(2\pi k \cdot) \rangle = \int_{-\frac{1}{2}}^{\frac{1}{2}} f(x) \cos(2\pi kx) \, dx
$$

Now we substitute the representation of $ f(x) $ into this integral:

$$
\langle f, \cos(2\pi k \cdot) \rangle = \int_{-\frac{1}{2}}^{\frac{1}{2}} \left( \frac{a_0}{2} + \sum_{j=1}^{n} a_j \cos(2\pi jx) + b_j \sin(2\pi jx) \right) \cos(2\pi kx) \, dx
$$


$$
\langle f, \cos(2\pi k \cdot) \rangle = \int_{-\frac{1}{2}}^{\frac{1}{2}} \frac{a_0}{2} \cos(2\pi kx) \, dx + \sum_{j=1}^{n} a_j \int_{-\frac{1}{2}}^{\frac{1}{2}} \cos(2\pi jx) \cos(2\pi kx) \, dx + \sum_{j=1}^{n} b_j \int_{-\frac{1}{2}}^{\frac{1}{2}} \sin(2\pi jx) \cos(2\pi kx) \, dx
$$

Using orthogonality properties:

1. **Orthogonality of cosine with cosine:**

$$
\int_{-\frac{1}{2}}^{\frac{1}{2}} \cos(2\pi jx) \cos(2\pi kx) \, dx = 
\begin{cases}
\frac{1}{2} & \text{if } j = k \\
0 & \text{if } j \neq k
\end{cases}
$$

2. **Orthogonality of sine with cosine:**

$$
\int_{-\frac{1}{2}}^{\frac{1}{2}} \sin(2\pi jx) \cos(2\pi kx) \, dx = 0 \quad \forall j, k
$$

This simplifies the expression for $ \langle f, \cos(2\pi k \cdot) \rangle $ to:

$$
\langle f, \cos(2\pi k \cdot) \rangle = \frac{a_k}{2}
$$

Where we have:

$$
a_k = 2 \langle f, \cos(2\pi k \cdot) \rangle = 2 \int_{-\frac{1}{2}}^{\frac{1}{2}} f(x) \cos(2\pi kx) \, dx
$$

for $ k = 0, 1, \ldots, n $.

The orthogonality property of the cosine functions allows all other terms in the series expansion of $ f(x) $ to cancel out, leaving only the $ a_k $ term.

Furthermore, to find $ b_k $, we take the inner product of $ f(x) $ with $ \sin(2\pi kx) $:

$$
\langle f, \sin(2\pi k \cdot) \rangle = \int_{-\frac{1}{2}}^{\frac{1}{2}} f(x) \sin(2\pi kx) \, dx
$$

Substituting the representation of $ f(x) $ into the integral:

$$
\langle f, \sin(2\pi k \cdot) \rangle = \int_{-\frac{1}{2}}^{\frac{1}{2}} \left( \frac{a_0}{2} + \sum_{j=1}^{n} a_j \cos(2\pi jx) + b_j \sin(2\pi jx) \right) \sin(2\pi kx) \, dx
$$

Expanding:

$$
\langle f, \sin(2\pi k \cdot) \rangle = \int_{-\frac{1}{2}}^{\frac{1}{2}} \frac{a_0}{2} \sin(2\pi kx) \, dx + \sum_{j=1}^{n} a_j \int_{-\frac{1}{2}}^{\frac{1}{2}} \cos(2\pi jx) \sin(2\pi kx) \, dx + \sum_{j=1}^{n} b_j \int_{-\frac{1}{2}}^{\frac{1}{2}} \sin(2\pi jx) \sin(2\pi kx) \, dx
$$

Using orthogonality properties:

1. **Orthogonality of cosine with sine:**

$$
\int_{-\frac{1}{2}}^{\frac{1}{2}} \cos(2\pi jx) \sin(2\pi kx) \, dx = 0 \quad \forall j, k
$$

2. **Orthogonality of sine with sine:**

$$
\int_{-\frac{1}{2}}^{\frac{1}{2}} \sin(2\pi jx) \sin(2\pi kx) \, dx = 
\begin{cases}
\frac{1}{2} & \text{if } j = k \\
0 & \text{if } j \neq k
\end{cases}
$$

This simplifies the expression for $ \langle f, \sin(2\pi k \cdot) \rangle $ to:

$$
\langle f, \sin(2\pi k \cdot) \rangle = \frac{b_k}{2}
$$

So,

$$
b_k = 2 \langle f, \sin(2\pi k \cdot) \rangle = 2 \int_{-\frac{1}{2}}^{\frac{1}{2}} f(x) \sin(2\pi kx) \, dx
$$

for $ k = 1, \ldots, n $.

Here the orthogonality property of the sine functions allows all other terms in the series expansion of $ f(x) $ to drop out, leaving only the $ b_k $ term.

Hence, we have shown that the Fourier coefficients $ a_k $ and $ b_k $ can be computed as:

$$
a_k = 2 \int_{-\frac{1}{2}}^{\frac{1}{2}} f(x) \cos(2\pi kx) \, dx, \quad k = 0, 1, \ldots, n
$$

$$
b_k = 2 \int_{-\frac{1}{2}}^{\frac{1}{2}} f(x) \sin(2\pi kx) \, dx, \quad k = 1, \ldots, n
$$

$\square$

#### e)

Using equidistant points $ x_0, \ldots, x_{N-1} $, where $ x_j = \frac{j}{N} $, $ j = 0, \ldots, N-1 $, for some $ N \in \mathbb{N} $, I want to approximate the integral required for the Fourier coefficients $c_k(f)$ of a function $f$. Note that the notation $ f_j = f(x_j) $ and $ \mathbf{f} = (f_0, \ldots, f_{N-1}) $ holds for simplicity.

The Fourier coefficient $c_k(f)$ is defined as:

$$
c_k(f) = \int_{0}^{1} f(x) e^{-2\pi i k x} \, dx.
$$

The trapezoidal rule is a numerical integration method that approximates the area under a curve by dividing it into trapezoids. To approximate this integral, we will use the composite trapezoidal rule with equidistant points $ x_j = \frac{j}{N} $, where $ j = 0, 1, \ldots, N-1 $.

The trapezoidal rule approximation is given by:

$$
c_k(f) \approx \hat{f}_k = \frac{1}{N} \sum_{j=0}^{N-1} f_j e^{-2\pi i jk / N}
$$

where $ f_j = f(x_j) = f\left(\frac{j}{N}\right) $.

Now we want to show $N$-Periodicity of $ \hat{f}_k $

In other words, we want to show that:

$$
\hat{f}_{k+N} = \hat{f}_k.
$$

Substituting $k+N$ into the formula for $ \hat{f}_k $:

$$
\hat{f}_{k+N} = \frac{1}{N} \sum_{j=0}^{N-1} f_j e^{-2\pi i j(k+N) / N}.
$$

Simplifying the exponent:

$$
\hat{f}_{k+N} = \frac{1}{N} \sum_{j=0}^{N-1} f_j e^{-2\pi i jk / N} e^{-2\pi i j}.
$$

Since $ e^{-2\pi i j} = 1 $ for any integer $ j $, we have:

$$
\hat{f}_{k+N} = \frac{1}{N} \sum_{j=0}^{N-1} f_j e^{-2\pi i jk / N} = \hat{f}_k.
$$

This shows that the coefficients $ \hat{f}_k $ are $ N $-periodic, meaning that:

$$
\hat{f}_{k+N} = \hat{f}_k \quad \forall k \in \mathbb{Z}.
$$

The periodicity of $ \hat{f}_k $ implies that the DFT, which uses these coefficients, assumes the original function $ f(x) $ is periodic with period $ 1 $ (the interval $[0, 1)$). If $ f(x) $ is not truly periodic with period $ 1 $, the periodicity of the DFT coefficients can cause *aliasing*, where higher frequencies in the original function are misrepresented in the approximation. The approximation of the Fourier coefficients using $ \hat{f}_k $ involves a truncation error due to the finite number of points $ N $. As $ N $ increases, this error decreases, and the approximation becomes more accurate.

#### f)

#### g)

#### h)

#### i)

#### j)

#### Task 2: Signal Processing

#### a)

#### b)

#### c)

#### d)

#### e)

#### Task 3: Sound Processing

#### a)

#### b)

#### c)

#### d)

#### e)

#### f)

#### g)